In [4]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import dataset
import schedulers

import tensorflow as tf
assert tf.config.list_physical_devices('GPU')

In [102]:
BATCH_SIZE = 32

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)

def build_model(EMBED_DIM=28, UNITS=253):  # EMBED_DIM=62, UNITS=252  == 2^20
    layer = input_text = keras.Input(batch_shape=(None, None), batch_size=BATCH_SIZE)
    layer = layers.Embedding(LETTERS_SIZE, EMBED_DIM, input_length=None, mask_zero=True)(layer)
    
    layer = layers.Dense(UNITS, activation=None)(layer)
    
    bidi = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.0), merge_mode='sum')
    layer = bidi(layer)
    layer = layers.add([layer, bidi(layer)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    model = keras.Model(inputs=input_text, outputs=outputs)

    jsmodel = model 
    return model, jsmodel

model, jsmodel = build_model()

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model_44"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_46 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_45 (Embedding)        (None, None, 28)     1232        input_46[0][0]                   
__________________________________________________________________________________________________
dense_177 (Dense)               (None, None, 253)    7337        embedding_45[0][0]               
__________________________________________________________________________________________________
bidirectional_51 (Bidirectional (None, None, 253)    1026168     dense_177[0][0]                  
                                                                 bidirectional_51[0][0]    

In [28]:
def accuracy(y_true, y_pred):
    K = keras.backend
    f = K.floatx()
    # convert dense predictions to labels
    y_pred_labels =  K.cast(K.argmax(y_pred, axis=-1), f)
    
    res = K.cast(K.equal(y_true, y_pred_labels), f)
    return K.sum(res) / K.sum(K.cast(K.not_equal(y_true, 0), f))


def fit(train_validation, scheduler=None, verbose=1, lr=1e-4):
    train, valid = train_validation
    model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=lr), metrics=[accuracy])
    callbacks = []
    if isinstance(scheduler, schedulers.CircularLearningRate):
        scheduler.set_dataset(train, BATCH_SIZE)
    if scheduler:
        callbacks.append(scheduler)
        
    x  = train.normalized
    vx = valid.normalized
    
    y  = {'N': train.niqqud, 'D': train.dagesh, 'S': train.sin }
    vy = {'N': valid.niqqud, 'D': valid.dagesh, 'S': valid.sin }
    
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=1, verbose=verbose, callbacks=callbacks)


MAXLEN = 82
def load_data(source, maxlen=MAXLEN, validation=0.1):
    filenames = [os.path.join('texts', f) for f in source]
    train, valid = dataset.load_data(filenames, validation, maxlen=maxlen)
    return train, valid

In [34]:
data_mix = load_data(['poetry', 'rabanit', 'pre_modern'])

In [144]:
model.load_weights('./checkpoints/uninit')
history = fit(data_mix, scheduler=schedulers.CircularLearningRate(30e-4, 80e-4, 1e-4)) #  val_N_accuracy: 0.9367 - val_D_accuracy: 0.9790 - val_S_accuracy: 0.9984
model.save_weights('./checkpoints/mix')

In [139]:
data_modern = load_data(validation=0.1, source=['modern'])

In [ ]:
model.load_weights('./checkpoints/mix')
history = fit(data_modern, scheduler=schedulers.CircularLearningRate(5e-3, 6e-3, 1e-4))  #  EMBED_DIM=28, UNITS=253: val_N_accuracy: 0.9575 - val_D_accuracy: 0.9856 - val_S_accuracy: 0.9994
model.save_weights('./checkpoints/modern')
model.save('models/modern.h5')

In [90]:
model.load_weights('./checkpoints/modern')
model.save('modern.h5')
import tensorflowjs as tfjs
# jsmodel.load_weights('./checkpoints/modern')
tfjs.converters.save_keras_model(jsmodel, '.')

In [125]:
model.load_weights('./checkpoints/modern')

def print_predictions(data, s):
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud[s], data.dagesh[s], data.sin[s]]
    actual = dataset.merge(data.text[s], ts=batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(data.text[s], ts=batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    total = []
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)
        last = expected_niqqud[i].tolist().index(0)
        res = expected_niqqud[i][:last] == actual_niqqud[i][:last]
        total.extend(res)
        print(round(np.mean(res), 2), f'({last - sum(res)} out of {last})')
        print()
    print(round(np.mean(total), 3))

print_predictions(data_modern[1], slice(0, None))

מצוי:  כְּלַבְלָב לֹא נִרְאֶה בָּאֶפֶק. וְזֶה לֹא שֶׁמִּיכֵל שָׁכְחָה, אֲבָל הָיָה נִרְאֶה שֶׁבְּכָל פַּעַם אִמָּא עֲסוּקָּה אוֹ לֹא 
רצוי:  כְּלַבְלַב לֹא נִרְאָה בָּאֹפֶק. וְזֶה לֹא שֶׁמִּיכַל שָׁכְחָה, אֲבָל הָיָה נִרְאֶה שֶׁבְּכָל פַּעַם אִמָּא עֲסוּקָה אוֹ לֹא 
0.95 (4 out of 77)

מצוי:  הַכְּפָר לִרְאוֹת אוֹתוֹ מִתְקָרֵב. הֵם הָיוּ בְּטוּחִים שֶׁהַמַּחְזָר הֶחָדָשׁ שֶׁל הַנַּעֲרָה נִכְשַׁל בַּמִּבְחַן הָאַחֲרוֹן, כְּמוֹ 
רצוי:  הַכְּפָר לִרְאוֹת אוֹתוֹ מִתְקָרֵב. הֵם הָיוּ בְּטוּחִים שֶׁהַמְּחַזֵר הֶחָדָשׁ שֶׁל הַנַּעֲרָה נִכְשַׁל בַּמִּבְחָן הָאַחֲרוֹן, כְּמוֹ 
0.95 (4 out of 81)

מצוי:  עֲצֶרֶת הַזְדַּהוּת עִם חַיָּילֵי צַהַ"ל וּמַעֲקֵב אַחַר הַהֲסָתָה בַּתִקְשׁוֹרֶת יְמָנִית. מְאוּחָר יוֹתֵר אוֹרְגָּנָה עֶצֶרֶת 
רצוי:  עֲצֶרֶת הִזְדַּהוּת עִם חַיָּילֵי צַהַ"ל וּמַעֲקָב אַחַר הַהֲסָתָה בְּתִקְשׁוֹרֶת יְמָנִית. מְאוּחָר יוֹתֵר אוּרְגְּנָה עֲצֶרֶת 
0.92 (6 out of 80)

מצוי:  הַצָּגָה גְּרָפִית שֶׁל הַהֶרֶס שֶׁגָּרְמָה הַפְצָצָה לָעִיר, אֶת סַבְלָם שֶׁל הַפְּצוּעִים וְאֶת הִתְמוֹד

In [ ]:
for i in range(50):
    model.load_weights('./checkpoints/pre_modern')
    p1 = np.exp(np.random.uniform(low=np.log(1e-5), high=np.log(1e-2)))
    p2 = np.exp(np.random.uniform(low=np.log(1e-4), high=np.log(1e-1)))
    p3 = np.exp(np.random.uniform(low=np.log(1e-5), high=np.log(1e-2)))
    print(p1, p2, p3, end=', ', sep=', ')
    history = fit(data_modern, scheduler=schedulers.CircularLearningRate(p1, p2, p3), verbose=0)
    print(history.history['val_N_accuracy'][0])

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()